In [1]:
import sys
import os
from pprint import pprint
sys.path.append(os.path.abspath("../../../"))
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display, HTML
from IPython.display import Markdown
import ipywidgets as iw
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
import usure.common.logging as usurelogging
from usure.config import config
from usure.classification.infrastructure import (
     BasicSentenceCleaner 
    ,FileLabeledCommentsDao
    ,FileWordVectorsRep)
from usure.classification.core import SvmLab, ClassifierInput, LabReport
from usure.classification.infrastructure import FileModelDao
import usure.classification.ui.utils as ui

Using TensorFlow backend.


In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = lines => false

<IPython.core.display.Javascript object>

In [4]:
pd.set_option('colheader_justify', 'left')
pd.set_option('display.max_colwidth', 200)

In [5]:
cleaner = BasicSentenceCleaner(config.assets)
comments_dao = FileLabeledCommentsDao(config.classification, cleaner)
wv_rep = FileWordVectorsRep(config.embeddings)
model_dao = FileModelDao(config.models)
comments = comments_dao.get("intertass-CR-train-tagged.xml")

In [6]:
def classification_lab_report(word_vector_no, word_vector_name, lab):
    display(Markdown(f'## {word_vector_no}. Creating models with embeddings: {word_vector_name}'))
    lab_report = lab.train_by_stratifiedkfold(folds=10)
    return lab_report

def model_architecture_summary(lab_report):
    display(Markdown(f'### Mean Accuracies and losses'))
    display(lab_report.summary)
    display(Markdown(f'### Accuracies and losses by Stratified KFold (folds with balanced class distributions)'))
    model_reports_df = ui.model_reports_to_DataFrame(lab_report.model_reports)
    display(model_reports_df) 
    
def show_statistics_by_model(word_vector_no, word_vector_name, lab_report):
    """Shows statistics by models created by every folds"""
    display(Markdown(f'## {word_vector_no}. {word_vector_name}'))
    for fold_no, model_report in enumerate(lab_report.model_reports):
        display(Markdown(f'### Fold number: {fold_no+1}. Name: {model_report.name}'))
        ui.plot_learning_curves_if_epoches(model_report)
        display(Markdown(f'### Training'))
        print(model_report.training.classification_report)
        confu_mtx = model_report.training.confusion_matrix
        display(confu_mtx)
        display(Markdown(f'### Validation'))
        print(model_report.validation.classification_report)
        confu_mtx = model_report.validation.confusion_matrix
        display(confu_mtx)

In [7]:
#%matplotlib notebook
display(Markdown(f'# Support Vector Machine'))
word_vectors = wv_rep.get_all()
word_vector_names = []
lab_reports = []
#word_vectors = [word_vectors.__next__()] #comment!!!!!
for i, word_vector in enumerate(word_vectors):    
    lab = SvmLab(ClassifierInput(comments, word_vector), model_dao)
    lab_report = classification_lab_report(i+1, word_vector.name, lab)
    model_architecture_summary(lab_report)
    word_vector_names.append(word_vector.name)
    lab_reports.append(lab_report)
    
for i, word_vector_name in enumerate(word_vector_names): 
    lab_report = lab_reports[i]
    show_statistics_by_model(i+1, word_vector_name, lab_report)
    

# Convolutional Neural Network

## 1. Creating models with embeddings: CorpusFBCR2013.txt.usu.bw.kvs

### Mean Accuracies and losses

,tra_acc_mean,val_acc_mean,tra_acc_stdev,val_acc_stdev
0,0.882501,0.402529,0.000546,0.007411


### Accuracies and losses by Stratified KFold (folds with balanced class distributions)

...

              precision    recall  f1-score   support

           N       0.77      1.00      0.87       280
         NEU       0.00      0.00      0.00        85
        NONE       0.99      1.00      1.00       149
           P       1.00      1.00      1.00       207

   micro avg       0.88      0.88      0.88       721
   macro avg       0.69      0.75      0.72       721
weighted avg       0.79      0.88      0.83       721



,N,NEU,NONE,P
Pred/True,,,,
N,280,0,0,0
NEU,84,0,1,0
NONE,0,0,149,0
P,0,0,0,207


### Validation

              precision    recall  f1-score   support

           N       0.40      1.00      0.57        31
         NEU       0.00      0.00      0.00         9
        NONE       1.00      0.06      0.12        16
           P       0.00      0.00      0.00        23

   micro avg       0.41      0.41      0.41        79
   macro avg       0.35      0.27      0.17        79
weighted avg       0.36      0.41      0.25        79



,N,NEU,NONE,P
Pred/True,,,,
N,31,0,0,0
NEU,9,0,0,0
NONE,15,0,1,0
P,23,0,0,0
